In [80]:
import pandas as pd
import hvplot.pandas
import requests
import json
from api_keys import geoapify_key

In [155]:
philly_crime_url = 'https://phl.carto.com/api/v2/sql?q=SELECT * FROM arrests_zipcode'
philly_crime = requests.get(philly_crime_url).json()
philly_crime = pd.DataFrame(philly_crime['rows'])
philly_crime = philly_crime.loc[philly_crime['zip_code'] != 'Unknown',:]
# philly_crime = philly_crime.sample(100000)
philly_crime

,cartodb_id,the_geom,the_geom_webmercator,offense_category,day,zip_code,defendant_race,count,objectid
0,1,None,None,Aggravated Assault,2011-01-13T05:00:00Z,19121,Black,2,26227634
1,2,None,None,Aggravated Assault,2011-01-13T05:00:00Z,19130,Black,2,26227635
2,3,None,None,Aggravated Assault,2011-01-13T05:00:00Z,19131,Black,1,26227636
3,4,None,None,Aggravated Assault,2011-01-13T05:00:00Z,19132,Black,1,26227637
4,5,None,None,Aggravated Assault,2011-01-13T05:00:00Z,19143,Black,1,26227638
...,...,...,...,...,...,...,...,...,...
376993,376994,None,None,Violation of Protection Order,2020-05-02T04:00:00Z,19149,White,1,26603104
376994,376995,None,None,Violation of Protection Order,2020-05-03T04:00:00Z,19116,White,2,26603105
376995,376996,None,None,Violation of Protection Order,2020-05-04T04:00:00Z,19119,Black,1,26603106
376996,376997,None,None,Violation of Protection Order,2020-05-05T04:00:00Z,19118,Black,1,26603107


In [156]:
philly_crime = philly_crime.loc[:,('zip_code','count')]
philly_crime_counts = pd.DataFrame(philly_crime['zip_code'].value_counts())
philly_crime_counts = philly_crime_counts.rename(columns={'zip_code':'offense_count'})
philly_crime_counts

,offense_count
19134,36950
19140,21032
19124,19892
19133,18365
19132,14620
19139,14573
19143,13962
19121,11353
19120,11286
19148,10765


In [157]:
philly_crime_counts['lat'] = ''
philly_crime_counts['lon'] = ''
for index, row in philly_crime_counts.iterrows():
    zipcode = index
    zipcodeurl = f'https://api.geoapify.com/v1/geocode/search?text={zipcode}&lang=en&limit=10&type=postcode&filter=countrycode:us&apiKey={geoapify_key}'
    zipcodecoords = requests.get(zipcodeurl).json()
    print(f'Converting zip code {zipcode} to coordinates')
    try:
        lat = zipcodecoords['features'][0]['properties']['lat']
        lon = zipcodecoords['features'][0]['properties']['lon']
        philly_crime_counts.loc[index,'lat'] = lat
        philly_crime_counts.loc[index,'lon'] = lon
        print(lat,lon)
    except:
        print('Bad item')
        lat = ''
        lon = ''
        pass


Converting zip code 19134 to coordinates
39.992856215 -75.11130586
Converting zip code 19140 to coordinates
40.009617474 -75.146059009
Converting zip code 19124 to coordinates
40.01523299 -75.088784716
Converting zip code 19133 to coordinates
39.988751517 -75.1420438
Converting zip code 19132 to coordinates
39.993517886 -75.164945773
Converting zip code 19139 to coordinates
39.96101535 -75.221072151
Converting zip code 19143 to coordinates
39.943270181 -75.222942213
Converting zip code 19121 to coordinates
39.978775174 -75.170031374
Converting zip code 19120 to coordinates
40.028970779 -75.121499097
Converting zip code 19148 to coordinates
39.919157966 -75.157547605
Converting zip code 19104 to coordinates
39.957497248 -75.197276366
Converting zip code 19131 to coordinates
39.981008869 -75.222526451
Converting zip code 19144 to coordinates
40.034325874 -75.172083913
Converting zip code 19141 to coordinates
40.035783215 -75.144103097
Converting zip code 19145 to coordinates
39.923856337

In [168]:
themap = philly_crime_counts.hvplot.points("lon","lat",geo = True,tiles = "CartoDark",frame_width = 1000,frame_height = 600, size = 'offense_count')

In [169]:
themap

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]   (offense_count)

In [160]:
philly_crime_counts

,offense_count,lat,lon
19134,36950,39.992856,-75.111306
19140,21032,40.009617,-75.146059
19124,19892,40.015233,-75.088785
19133,18365,39.988752,-75.142044
19132,14620,39.993518,-75.164946
19139,14573,39.961015,-75.221072
19143,13962,39.94327,-75.222942
19121,11353,39.978775,-75.170031
19120,11286,40.028971,-75.121499
19148,10765,39.919158,-75.157548
